In [1]:
import pandas as pd
import numpy as np

Load data from csv and look at column labels and data

In [45]:
hosp = pd.read_csv('pay_value_hosp.csv')
hosp.head()

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Payment Measure ID,Payment Measure Name,...,Payment,Lower Estimate,Higher Estimate,Payment Footnote,Value of Care Display ID,Value of Care Display Name,Value of Care Category,Value of Care Footnote,Start Date,End Date
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_AMI,Payment for heart attack patients,...,"$24,934","$23,490","$26,429",NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019
1,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_HF,Payment for heart failure patients,...,"$17,854","$17,061","$18,676",NaN,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019
2,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_PN,Payment for pneumonia patients,...,"$20,216","$19,186","$21,271",NaN,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment,NaN,07/01/2016,06/30/2019
3,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,"$22,216","$21,191","$23,313",NaN,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment,NaN,04/01/2016,03/31/2019
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,PAYM_30_AMI,Payment for heart attack patients,...,"$24,742","$22,016","$27,967",NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019


Look at column 'Payment Measure ID', notice consistency - each hospital has 4 IDs. Suggets 4710 hospitals. Perhaps group dataset by one hospital per row?

In [46]:
hosp.value_counts('Payment Measure ID')

Payment Measure ID
PAYM_30_AMI         4710
PAYM_30_HF          4710
PAYM_30_PN          4710
PAYM_90_HIP_KNEE    4710
dtype: int64

In [47]:
hosp_columns = hosp.columns
hosp_columns = hosp_columns.drop(['Facility ID', 'Facility Name', 'Address', 'City', 'ZIP Code', 'County Name', 'Phone Number', 'State', 'Payment Measure ID', 'Payment Measure Name'])

Helper functions to clean data before converting to csv file

In [111]:
def iterate_first_row(cols, df): 
    i = 0
    new_data = {}
    for col in cols: 
        col = "Heart attack " + col
    while i < 22: 
        new_data[cols[i]] = df.iloc[0 ,i]
        i += 1
    new_df = pd.DataFrame(new_data, index=[df['Facility ID']], columns=cols)
    return new_df.drop_duplicates()
        
def iterate_next_row(cols, df, row):
    i = 8
    new_data = {}
    for col in cols:
        col = "Heart failure " + col
    while i < 22: 
        new_data[cols[i]] = df.iloc[row, i]
        i += 1
    new_df = pd.DataFrame(new_data, index=[df['Facility ID']], columns=cols)
    return new_df.drop_duplicates()
    
def clean(df): 
    new_df = iterate_first_row(df.columns, df)
    second_row_df = iterate_next_row(df.columns, df, 1)
    together =  new_df.join(second_row_df, lsuffix=' Heart attack', rsuffix=" Heart Failure")
    third_row_df = iterate_next_row(df.columns, df, 2)
    together = together.join(third_row_df)
    fourth_row_df = iterate_next_row(df.columns, df, 3)
    together = together.join(fourth_row_df, lsuffix=' Pneumonia', rsuffix=" Hip/Knee Replacement")
    return together

See example of GroupBy object

In [112]:
print(hosp.columns)
hosp.groupby(by='Facility ID').get_group(10001)


Index(['Facility ID', 'Facility Name', 'Address', 'City', 'State', 'ZIP Code',
       'County Name', 'Phone Number', 'Payment Measure ID',
       'Payment Measure Name', 'Payment Category', 'Denominator', 'Payment',
       'Lower Estimate', 'Higher Estimate', 'Payment Footnote',
       'Value of Care Display ID', 'Value of Care Display Name',
       'Value of Care Category', 'Value of Care Footnote', 'Start Date',
       'End Date'],
      dtype='object')


,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Payment Measure ID,Payment Measure Name,...,Payment,Lower Estimate,Higher Estimate,Payment Footnote,Value of Care Display ID,Value of Care Display Name,Value of Care Category,Value of Care Footnote,Start Date,End Date
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_AMI,Payment for heart attack patients,...,"$24,934","$23,490","$26,429",NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019
1,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_HF,Payment for heart failure patients,...,"$17,854","$17,061","$18,676",NaN,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019
2,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_PN,Payment for pneumonia patients,...,"$20,216","$19,186","$21,271",NaN,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment,NaN,07/01/2016,06/30/2019
3,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,"$22,216","$21,191","$23,313",NaN,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment,NaN,04/01/2016,03/31/2019


Call helper function to reformat code

In [110]:
new_hosp = hosp.groupby(by='Facility ID').apply(clean)
new_hosp

,,Facility ID Heart attack,Facility Name Heart attack,Address Heart attack,City Heart attack,State Heart attack,ZIP Code Heart attack,County Name Heart attack,Phone Number Heart attack,Payment Measure ID Heart attack,Payment Measure Name Heart attack,...,Payment Hip/Knee Replacement,Lower Estimate Hip/Knee Replacement,Higher Estimate Hip/Knee Replacement,Payment Footnote Hip/Knee Replacement,Value of Care Display ID Hip/Knee Replacement,Value of Care Display Name Hip/Knee Replacement,Value of Care Category Hip/Knee Replacement,Value of Care Footnote Hip/Knee Replacement,Start Date Hip/Knee Replacement,End Date Hip/Knee Replacement
Facility ID,Facility ID,,,,,,,,,,,,,,,,,,,,,
10001,10001,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_AMI,Payment for heart attack patients,...,"$22,216","$21,191","$23,313",NaN,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment,NaN,04/01/2016,03/31/2019
10005,10005,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,PAYM_30_AMI,Payment for heart attack patients,...,"$20,679","$19,691","$21,772",NaN,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Average Payment,NaN,04/01/2016,03/31/2019
10006,10006,10006,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,FLORENCE,AL,35630,LAUDERDALE,(256) 768-8400,PAYM_30_AMI,Payment for heart attack patients,...,"$24,089","$23,098","$25,097",NaN,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment,NaN,04/01/2016,03/31/2019
10007,10007,10007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,(334) 493-3541,PAYM_30_AMI,Payment for heart attack patients,...,Not Available,Not Available,Not Available,1.0,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Not Available,13.0,04/01/2016,03/31/2019
10008,10008,10008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW,(334) 335-3374,PAYM_30_AMI,Payment for heart attack patients,...,Not Available,Not Available,Not Available,5.0,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Not Available,13.0,04/01/2016,03/31/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670133,670133,670133,TRUSTED MEDICAL CENTERS MANSFIELD,1710 N HWY 287 FRONTAGE ROAD,MANSFIELD,TX,76063,TARRANT,(817) 225-4809,PAYM_30_AMI,Payment for heart attack patients,...,Not Available,Not Available,Not Available,19.0,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Not Available,19.0,04/01/2016,03/31/2019
670134,670134,670134,ALTUS LUMBERTON HOSPITAL,220 BLANCO ST,LUMBERTON,TX,77657,HARDIN,(409) 227-0520,PAYM_30_AMI,Payment for heart attack patients,...,Not Available,Not Available,Not Available,5.0,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Not Available,5.0,04/01/2016,03/31/2019
670135,670135,670135,"ALTUS HOUSTON HOSPITAL, LP",6011 W SAM HOUSTON PKWY SOOUTH,HOUSTON,TX,77072,HARRIS,(713) 773-0556,PAYM_30_AMI,Payment for heart attack patients,...,Not Available,Not Available,Not Available,19.0,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Not Available,19.0,04/01/2016,03/31/2019


Look at columns info - will need to make metadata file 

In [114]:
new_hosp.columns

Index(['Facility ID Heart attack', 'Facility Name Heart attack',
       'Address Heart attack', 'City Heart attack', 'State Heart attack',
       'ZIP Code Heart attack', 'County Name Heart attack',
       'Phone Number Heart attack', 'Payment Measure ID Heart attack',
       'Payment Measure Name Heart attack', 'Payment Category Heart attack',
       'Denominator Heart attack', 'Payment Heart attack',
       'Lower Estimate Heart attack', 'Higher Estimate Heart attack',
       'Payment Footnote Heart attack',
       'Value of Care Display ID Heart attack',
       'Value of Care Display Name Heart attack',
       'Value of Care Category Heart attack',
       'Value of Care Footnote Heart attack', 'Start Date Heart attack',
       'End Date Heart attack', 'Facility ID Heart Failure',
       'Facility Name Heart Failure', 'Address Heart Failure',
       'City Heart Failure', 'State Heart Failure', 'ZIP Code Heart Failure',
       'County Name Heart Failure', 'Phone Number Heart Failure',

In [115]:
new_hosp.to_csv('hospitals.csv', index=False, header=False)